# Imports and Configurations

In [ ]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

np.set_printoptions(precision=3, suppress=True)
ecm = 'mean_absolute_error'
# ecm = 'mean_squared_error'
k_fold_value = 5

raw_dataset = pd.read_csv('dataset_v1.csv', sep=',', skipinitialspace=True)
dataset = raw_dataset.copy()
predict_set = dataset.tail(20)
dataset = dataset.drop(predict_set.index)

predictors = ['SampleNo', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6']
all_test_results = {}

# Functions

In [ ]:
# Utility Classes
class ModelResult:
    def __init__(self, info_dict: dict):
        self.model = info_dict['model']
        self.arg = info_dict['arg']
        self.predictors = info_dict['predictors']
        if 'error' not in info_dict.keys():
            self.y_actual = info_dict['y_actual']
            self.y_predicted = info_dict['y_predicted']
            self.error = ModelResult.calculate_error(y_act=self.y_actual, y_pred=self.y_predicted)
        else:
            self.y_actual = None
            self.y_predicted = None
            self.error = info_dict['error']

    def display_y_comparison(self, transpose=True):
        y_comp = pd.DataFrame({'actual': list(self.y_actual), 'predicted': self.y_predicted})
        if transpose:
            y_comp = y_comp.transpose()
        return display(y_comp)

    @staticmethod
    def calculate_error(y_act, y_pred):
        def calculate_mae():
            test_error = 0
            for actual, predicted in zip(y_act, y_pred):
                test_error += abs(actual - predicted)
            return test_error / len(y_pred)

        def calculate_mse():
            test_error = 0
            for actual, predicted in zip(y_act, y_pred):
                test_error += (actual - predicted) ** 2
            return test_error / len(y_pred)

        if ecm == 'mean_absolute_error':
            return calculate_mae()
        elif ecm == 'mean_squared_error':
            return calculate_mse()
        else:
            print("You can't reach here.")


class ModelResultsTable:
    def __init__(self):
        self.all_model_results = {}  # {'mul_lin_reg': [ModelResult0, ModelResult1, ...], 'lin_reg': [...] ... }
        self.model_results_with_best_params = []

    def add_model(self, model_results: list):
        sorted_models = list(sorted(model_results, key=lambda item: item.error))
        best_performing_var = sorted_models[0]
        self.all_model_results[best_performing_var.model] = sorted_models
        self.model_results_with_best_params.append(best_performing_var)
        self.model_results_with_best_params = list(sorted(self.model_results_with_best_params,
                                                          key=lambda item: item.error))

    def display_error_table(self, transpose=True):
        error_table = pd.DataFrame([[result.model, result.arg, result.predictors, result.error]
                                    for result in self.model_results_with_best_params],
                                   columns=['model', 'args', 'predictors', ecm])
        if transpose:
            error_table = error_table.transpose()
        display(error_table)


all_results_table = ModelResultsTable()


# Utility Functions
def k_fold_sampling(sample_source=dataset):
    kf = KFold(n_splits=k_fold_value, random_state=None, shuffle=False)

    features = np.array(sample_source[predictors])
    labels = np.array(sample_source['Y'])
    k_s = []
    for train_i, test_i in kf.split(dataset):
        k_s.append({'train_features': pd.DataFrame(features[train_i], columns=predictors),
                    'train_labels': labels[train_i],
                    'test_features': pd.DataFrame(features[test_i], columns=predictors),
                    'test_labels': labels[test_i]})
    return k_s


def k_fold_cv(samples, reg_func, arg=None, inputs=predictors):
    k_fold_cv_mae = 0
    first_fold_prediction = None
    if arg is not None:
        arg_key, arg_value = arg[0], arg[1]
        arg_text = f'{arg_key}={arg_value}'
    else:
        arg_value = None
        arg_text = ''
    if type(inputs) != list:
        inputs = [inputs]
    for i in range(len(samples)):
        x_train = pd.DataFrame(samples[i]['train_features'][inputs], columns=inputs)
        x_test = pd.DataFrame(samples[i]['test_features'][inputs], columns=inputs)
        ki_result = reg_func(x_train=x_train, y_train=samples[i]['train_labels'],
                             x_test=x_test, y_test=samples[i]['test_labels'],
                             arg=arg_value, inputs=inputs)
        if first_fold_prediction is None:
            first_fold_prediction = ki_result.y_predicted
        k_fold_cv_mae += ki_result.error

    if reg_func == single_linear_regression:
        return ModelResult({'model': reg_func.__name__,
                            'arg': arg_text,
                            'predictors': inputs,
                            'error': k_fold_cv_mae / len(samples)}), first_fold_prediction
    return ModelResult({'model': reg_func.__name__,
                        'arg': arg_text,
                        'predictors': inputs,
                        'error': k_fold_cv_mae / len(samples)})


# Regression Functions
def regression(x_train, y_train, x_test):
    x_regressor = LinearRegression().fit(x_train, y_train)
    return x_regressor.predict(x_test)


def single_linear_regression(x_train, y_train, x_test, y_test, arg=None, inputs=None):
    y_predicted = regression(x_train=np.array(x_train), y_train=y_train,
                             x_test=np.array(x_test))
    # y_predicted = regression(x_train=np.array(x_train[inputs])[:, None], y_train=y_train,
    #                         x_test=np.array(x_test[inputs])[:, None])
    return ModelResult({'model': 'single_linear_regression',
                        'arg': arg,
                        'predictors': inputs,
                        'y_actual': y_test,
                        'y_predicted': y_predicted})


def multiple_linear_regression(x_train, y_train, x_test, y_test, arg, inputs=predictors):
    y_predicted = regression(x_train=x_train[inputs], y_train=y_train, x_test=x_test[inputs])
    return ModelResult({'model': 'multiple_linear_regression',
                        'arg': arg,
                        'predictors': inputs,
                        'y_actual': y_test,
                        'y_predicted': y_predicted})


def polynomial_regression(x_train, y_train, x_test, y_test, arg=2, inputs=predictors):
    poly_features = PolynomialFeatures(degree=arg, include_bias=False)
    y_predicted = regression(x_train=poly_features.fit_transform(x_train[inputs]), y_train=y_train,
                             x_test=poly_features.fit_transform(x_test[inputs]))
    return ModelResult({'model': 'polynomial_regression',
                        'arg': f'o={arg}',
                        'predictors': ', '.join(inputs),
                        'y_actual': y_test,
                        'y_predicted': y_predicted})


def k_nearest_regression(x_train, y_train, x_test, y_test, arg=3, inputs=predictors):
    def euclidean_dist(p_x, p_y):
        distances = list(map(lambda pr: (p_x[pr] - p_y[pr]) ** 2, predictors))
        return sum(distances)

    y_predicted = []
    for i in range(len(x_test.index)):
        current_row = x_test.iloc[i]
        current_distances = [[y_train[n], euclidean_dist(current_row, x_train.iloc[n])]
                             for n in range(len(x_train.index))]
        current_distances.sort(key=lambda n: n[1])
        k_nearest_avg = sum([n[0] for n in current_distances[:arg]]) // arg
        y_predicted.append(k_nearest_avg)

    return ModelResult({'model': 'k_nearest_regression',
                        'arg': f'k={arg}',
                        'predictors': ', '.join(inputs),
                        'y_actual': y_test,
                        'y_predicted': y_predicted})


def decision_tree_regression(x_train, y_train, x_test, y_test, arg=5, inputs=predictors):
    regressor = DecisionTreeRegressor(random_state=0, max_depth=arg).fit(x_train, y_train)
    y_predicted = regressor.predict(x_test)

    return ModelResult({'model': 'decision_tree_regression',
                        'arg': f'd={arg}',
                        'predictors': ', '.join(inputs),
                        'y_actual': y_test,
                        'y_predicted': y_predicted})

Before applying any machine learning algorithms, I wanted to see how each of the given predictors relate to each other.
To do it, I have found out pairplot() function from seaborn library which takes the name of columns (predictors) as
parameters and plots each one of them with another.

In [ ]:
sns.pairplot(dataset[predictors], diag_kind='kde')

From looking at those results, it is clear that x1 and x6 values are exactly the same. So, I have decided to drop x6
predictor since we do not need the same values twice. Also, I have previously included SampleNo as a predictor (even it is
not really logical) just to see if it makes the algorithms perform better, which it obviously did not. So, I am also
dropping the SampleNo from the dataset and the predictors list.

In [ ]:
dataset = dataset.drop('x6', axis=1)
predictors.remove('x6')

dataset = dataset.drop('SampleNo', axis=1)
predictors.remove('SampleNo')

Now, I wanted to see how each of those predictors relate to Y separately.

In [ ]:
fig_, axes_ = plt.subplots(2, 3, figsize=(18, 10))
axes_[1][2].set_visible(False)
axes_[1][0].set_position([0.24, 0.125, 0.228, 0.343])
axes_[1][1].set_position([0.55, 0.125, 0.228, 0.343])

for i_ in range(len(predictors)):
    sns.scatterplot(ax=axes_[i_ // 3, i_ % 3], x=dataset[predictors[i_]], y=dataset['Y'], alpha=0.5, color='b')

table_heuristics = dataset.describe().transpose()[['min', 'max', 'mean', 'std']]
display(table_heuristics)

From here, I can see that when x1 has low values, their corresponding Y values are also tend to be lower. Similarly,
when x5 has a value greater than 5, its corresponding Y value is lower. Because of this, I assume that some kind of
classification method must be applied.

In [ ]:
# from sklearn.preprocessing import StandardScaler, normalize
#
# scaled_data = StandardScaler().fit_transform(dataset[predictors])
# x_normalized = normalize(scaled_data, norm='l1', axis=0)
# dataset_normalized = pd.DataFrame(x_normalized, columns=predictors)
#
# dataset_normalized['Y'] = dataset['Y']
# dataset = dataset_normalized

k_samples = k_fold_sampling(dataset)

In [ ]:
fig_, axes_ = plt.subplots(2, 3, figsize=(18, 10))
axes_[1][2].set_visible(False)
axes_[1][0].set_position([0.24, 0.125, 0.228, 0.343])
axes_[1][1].set_position([0.55, 0.125, 0.228, 0.343])

for i_ in range(len(predictors)):
    sns.scatterplot(ax=axes_[i_ // 3, i_ % 3], x=dataset[predictors[i_]], y=dataset['Y'], alpha=0.5, color='b')

table_heuristics = dataset.describe().transpose()[['min', 'max', 'mean', 'std']]
display(table_heuristics)

# I. Linear Regression

## I.I. Single Linear Regression

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes[1][2].set_visible(False)
axes[1][0].set_position([0.24, 0.125, 0.228, 0.343])
axes[1][1].set_position([0.55, 0.125, 0.228, 0.343])

# lin_error_dict = {}
lin_reg_results = []
ax_i = 0
for p in predictors:
    ax = axes[ax_i // 3, ax_i % 3]

    k_fold_result, first_fold_pred = k_fold_cv(k_samples, single_linear_regression, inputs=p)
    lin_reg_results.append(k_fold_result)

    sns.scatterplot(ax=ax, x=k_samples[0]['test_features'][p], y=k_samples[0]['test_labels'],
                    alpha=0.7, color='b', label='Test Data')
    sns.scatterplot(ax=ax, x=k_samples[0]['train_features'][p], y=k_samples[0]['train_labels'],
                    alpha=0.3, color='g', label='Training Data')
    sns.regplot(ax=ax, x=k_samples[0]['test_features'][p], y=first_fold_pred,
                color='r', scatter=False, label='Predicted Y', truncate=False)

    ax.set_xlim(table_heuristics['min'][p], table_heuristics['max'][p])
    ax.set_xlabel(p)
    ax.set_ylabel('Y')
    ax.legend()
    ax_i += 1

all_results_table.add_model(lin_reg_results)

## I.II. Multiple Linear Regression

In [ ]:
import itertools

predictor_combinations = []
for L in range(2, len(predictors)+1):
    predictor_combinations.extend(itertools.combinations(predictors, L))

multi_lin_reg_args = list(map(list, predictor_combinations))
multi_lin_reg_results = [k_fold_cv(samples=k_samples, reg_func=multiple_linear_regression, inputs=ps)
                         for ps in multi_lin_reg_args]

all_results_table.add_model(multi_lin_reg_results)

## I.III. Polynomial Regression

In [ ]:
poly_reg_args = [2, 3, 4]
poly_reg_results = [k_fold_cv(samples=k_samples, reg_func=polynomial_regression, arg=['o', order])
                    for order in poly_reg_args]

all_results_table.add_model(poly_reg_results)

## I.IV. K-Nearest-Neighbors Regression

In [ ]:
knn_args = [2, 3, 4, 5]
k_nearest_results = [k_fold_cv(samples=k_samples, reg_func=k_nearest_regression, arg=['k', k]) for k in knn_args]

all_results_table.add_model(k_nearest_results)

## I.V. Decision Tree Regression

In [ ]:
dec_tree_args = range(1, 11)
dec_tree_results = [k_fold_cv(samples=k_samples, reg_func=decision_tree_regression, arg=['d', d])
                    for d in dec_tree_args]

all_results_table.add_model(dec_tree_results)
all_results_table.display_error_table(transpose=False)

# TESTING

In [ ]:
from sklearn.ensemble import RandomForestRegressor

def random_forest_regression(x_train, y_train, x_test, y_test, arg=None, inputs=predictors):
    if arg is None:
        arg = ['d', 2]
    regressor = RandomForestRegressor(max_depth=arg, random_state=0).fit(x_train, y_train)
    y_predicted = regressor.predict(x_test)

    return ModelResult({'model': 'random_forest_regression',
                        'arg': f'd={arg}',
                        'predictors': ', '.join(inputs),
                        'y_actual': y_test,
                        'y_predicted': y_predicted})

rand_forest_args = range(1, 11)
rand_forest_results = [k_fold_cv(samples=k_samples, reg_func=random_forest_regression,
                                 arg=['d', d], inputs=predictors) for d in rand_forest_args]
all_results_table.add_model(rand_forest_results)
all_results_table.display_error_table(transpose=False)


In [ ]:
%reload_ext watermark
%watermark -v -m -p sklearn,pandas,numpy,matplotlib,seaborn
%watermark -u -n -t -z